TODO Luki format??

Co prioritizovat? https://github.com/zdenkas/human-tafazzin-variants-database/blob/main/TODO.txt
Konzistence cisel (pozic) a nukleotidu
Jake k tomu pouzit data?
Kolik nekonzistenci tam najdeme? Pak to poslat BSF


In [ ]:
import pandas as pd
from Bio import SeqIO

In [ ]:
import helpers

In [ ]:
database_folder = '../database_versions'

In [ ]:
input_path_prefix = '../database_versions/0001_2023-10-30-21-51-49-628455_Human-TAFAZZIN-Variants-Database_' 
# TODO get this automatically in next notebooks

output_path_prefix = helpers.create_database_output_path_prefix()

In [ ]:
taz_cdna_path = '../data_external/Homo_sapiens_ENST00000601016_6_sequence.fa'
taz_positions_path = '../data_external/ExonsSpreadsheet-Homo_sapiens_Transcript_Exons_ENST00000601016_whole_table.csv'

### Load positions

in hg38

In [ ]:
taz_positions = pd.read_csv(taz_positions_path)

In [ ]:
taz_positions['Start'] = pd.to_numeric(taz_positions['Start'].str.replace(',', ''), errors='coerce')
taz_positions['Start'] = taz_positions['Start'].fillna(0).astype(int)

In [ ]:
taz_positions['End'] = pd.to_numeric(taz_positions['End'].str.replace(',', ''), errors='coerce')
taz_positions['End'] = taz_positions['End'].fillna(0).astype(int)

In [ ]:
taz_positions.head()

### Load cdna

In [ ]:
for seq_record in SeqIO.parse(taz_cdna_path, "fasta"):
    #print(seq_record.id)
    #print(seq_record.name)
    print(seq_record.description)
    #print(repr(seq_record.seq))
    #print(len(seq_record))
    print('-----')

In [ ]:
len(seq_record)

In [ ]:
seq_record.seq

In [ ]:
seq_record.seq[-1]

In [ ]:
seq_record

### Load Tafazzin database

In [ ]:
df_pathogenic = pd.read_csv(input_path_prefix + 'pathogenic.csv')
df_vus = pd.read_csv(input_path_prefix + 'vus.csv')
df_exon5 = pd.read_csv(input_path_prefix + 'exon5.csv')
df_benign = pd.read_csv(input_path_prefix + 'benign.csv')

In [ ]:
df_pathogenic.head(1)

In [ ]:
loc_38_col = 'Location in Genome release 38 (hg38)'
loc_37_col = 'Location in Genome release 37 (hg19)'
dna_modif_col = 'DNA Modifications'

In [ ]:
def get_exon_intron_from_taz_positions(taz_database_38_loc, taz_positions=taz_positions):
    matching_rows = taz_positions[(taz_positions['Start'] <= taz_database_38_loc) & (taz_positions['End'] >= taz_database_38_loc)]
    exon_intron_values = matching_rows['Exon / Intron'].values

    # If you expect only one match and want to get a single value
    if len(exon_intron_values) == 1:
        return exon_intron_values[0]
    else:
        print("Multiple or no matching rows found!")
        return None

In [ ]:
def find_seq_record(search_string, taz_cdna_path=taz_cdna_path):
    # Use a generator expression to find the first matching seq_record
    seq_record = next((record for record in SeqIO.parse(taz_cdna_path, "fasta") if search_string in record.description), None)
    return seq_record


In [ ]:
for key, row in df_pathogenic.iterrows():
    try:
        taz_database_38_loc = int(row[loc_38_col].split(':')[1])
        taz_database_37_loc = int(row[loc_37_col].split(':')[1])
        taz_database_dna_modif = row[dna_modif_col]
        exon_intron_from_taz_positions = get_exon_intron_from_taz_positions(taz_database_38_loc)
        seq_record = find_seq_record(exon_intron_from_taz_positions)

        if seq_record:
            print(f"Found record: {seq_record.description}")
            print(seq_record.sequence)
        else:
            print("No matching record found.")
        
    except Exception as ex:
        print(key, row[loc_38_col]) #, ex)
    break
        

In [ ]:
matching_record.seq

In [ ]:
taz_positions

In [ ]:
taz_positions.Start

In [ ]:
taz_positions.head(20)

In [ ]:
taz_database_38

In [ ]:
taz_database_37

In [ ]:
taz_database_dna_modif

In [ ]:
df_pathogenic.Location.value_counts()